<a id='em_topic_modelling'></a>
# Тематическое моделирование

![](http://imgur.com/S8WgwBp.png)

<a id='toc'></a>
# Содержание
* [1. Рассуждения на тему тематического моделирования](#discussion_on_topic_modeling)
* [2. Реализация ЕМ-алгоритма для модели PLSA и его применение](#plsa_and_application)
    * [2.1 Загрузка данных](#load)
    * [2.2 Реализация PLSA](#plsa)
    * [2.3 Задание 1 [3 балла]](#plsa_task1)
    * [2.4 Задание 2 [0.5 балла]](#plsa_task2)
    * [2.5 Задание 3 [0.5 балла]](#plsa_task3)
* [3. Модель LDA и визуализация](#lda_and_visualization)
    * [3.1 Задание 1 [1 балла]](#lda_task1)
    * [3.2 Задание 2 [1 балл]](#lda_task2)

In [ ]:
%autosave 30

import numpy as np
import pandas as pd
import copy
import sys
_add_to_path = True
import math
import scipy
import pickle as pkl
from scipy import stats
from scipy.special import erfc

from itertools import product, chain
from collections import defaultdict, Counter
from tqdm import tqdm

#matplotlib
import matplotlib
import matplotlib as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter
%matplotlib inline

#sklearn
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import CountVectorizer

matplotlib.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
matplotlib.rcParams['legend.handletextpad'] = 0.01
matplotlib.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
matplotlib.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.titlesize'] = 20

matplotlib.rc('xtick', labelsize=14)
matplotlib.rc('ytick', labelsize=14)
matplotlib.rc('legend', fontsize=16)

matplotlib.rc('font', **{'family':'serif'})
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', unicode=True)
matplotlib.rc('text.latex', preamble=r'\usepackage[utf8]{inputenc}')
matplotlib.rc('text.latex', preamble=r'\usepackage[english]{babel}') 
matplotlib.rcParams['axes.labelsize'] = 20

In [ ]:
if _add_to_path:
    sys.path.append('../../')
from ml.core import Checker, Printer
from ml.mixtures import *

%load_ext autoreload
%autoreload 1
# Below comes the list of modules which is automatically reimported

<a id='discussion_on_topic_modeling'></a>
# 1. Рассуждения на тему тематического моделирования [[toc](#toc)]

Тематическое моделирование является популярным инструментом анализа текстов. Задача заключается в поиске тем $T$, которые хорошо бы описывали документы $D$ со словарём $W$. Большинство тематических моделей оперирует данными в формате "мешка слов", т.е. учитывают только частоты слов в документах, а не их порядок. Одной из простейших тематических моделей является [PLSA](https://en.wikipedia.org/wiki/Probabilistic_latent_semantic_analysis), которая приводит к задаче стохастического матричного разложения: 

$$F \approx \Phi \times \Theta$$
где
- $F_{W \times D}$— матрица распределений слов в документах (нормированные частоты)
- $\Phi_{W \times T}$ — матрица распределений слов в темах (модель)
- $\Theta_{T \times D}$ — матрица распределений тем в документах (результат применения модели к обучающим данным)

Можно сказать, что алгоритмы тематического моделирования производят мягкую бикластеризацию данных:
 - *мягкую*, так как объекты относятся не строго к одному кластеру, а к нескольким с разными вероятностями
 - *бикластеризацию*, так как модель одновременно кластеризует слова по темам и темы по документам.

## 1.1 EM-алгоритм [[toc](#toc)]

![](http://imgur.com/EeIuI1T.png)

С вероятностной точки зрения, задача обучения модели PLSA ставится как максимизация неполного правдоподобия по параметам $\Phi$ и $\Theta$. ЕМ-алгоритм для модели PLSA заключается в повторении двух шагов:

- **Е-шаг** — оценка распределений тем для каждого слова в каждом документе по параметрам $\Phi$ и $\Theta$ (шаг 6);
- **М-шаг** — обновление параметров $\Phi$ и $\Theta$ на основе полученных оценок (шаги 7 и 9).

Существуют различные модификации итерационного процесса, позволяющие снизить расходы по памяти. В данном случае, мы избегаем хранения трехмерной матрицы $p_{tdw}$, сразу пересчитывая $\Theta$ для текущего документа и аккумулируя счетчики $n_{wt}$ для последующего пересчета $\Phi$.

Псевдокод алгритма записывается следующим образом:

1. Инициализировать $\phi_{wt}^0$ для всех $w \in W$, $t \in T$ и $\theta_{td}^0$ для всех $t \in T$, $d \in D$
2. Внешний цикл по итерациям $i = 1 ... max\_iter$:
3. $\quad$ $n_{wt}^i := 0$, $n_t^i := 0$ для всех $w \in W$ и $t \in T$ 
4. $\quad$ Внутренний цикл по документам $d \in D$  
5. $\qquad$ $Z_w := \sum_{t \in T} \phi_{wt}^{i-1}\theta_{td}^{i-1}$ для всех $w \in d$ $\cfrac{}{}$
6. $\qquad$ $p_{tdw} := \cfrac{ \phi_{wt}^{i-1}\theta_{td}^{i-1} }{ Z_w }$ (**E-шаг**)
7. $\qquad$ $\theta_{td}^{i} := \cfrac{ \sum_{w \in d} n_{dw} p_{tdw} }{ n_d }$ для всех $t \in T$ (**M-шаг**)
8. $\qquad$ Увеличить $n_{wt}^i$ и $n_t^i$ на $n_{dw} p_{tdw}$ для всех $w \in W$ и $t \in T$
9. $\quad \phi_{wt}^i := \cfrac{n_{wt}^i}{n_t^i}$ для всех $w \in W$ и $t \in T$ (**M-шаг**)

Обозначения:
 - $p_{tdw}$ — вероятность темы $t$ для слова $w$ в документе $d$
 - $\phi_{wt}$ — элемент матрицы $\Phi$, соответствующий вероятности слова $w$ в теме $t$
 - $\theta_{td}$ — элемент матрицы $\Theta$, соответствующий вероятности темы $t$ в документе $d$
 - $n_{wt}$ — элемент матрицы счётчиков отнесения слова $w$ к теме $t$ (путем нормирования этой матрицы получается матрица $\Phi$)
 - $Z_w$ — элемент вектора вспомогательных переменных, соответствующий слову $w$
 - $n_t$ — вектор нормировочных констант для матрицы $n_{wt}$
 - $n_d$ — вектор нормировочных констант для матрицы $n_{dw}$
 - $n$ — суммарное число слов в коллекции

## 1.2  Оценка качества [[toc](#toc)]

Для оценивания качества построенной модели и контроля сходимости процесса обучения обычно используют [перплексию](http://www.machinelearning.ru/wiki/images/8/88/Voron-iip9-talk.pdf):

$$\mathcal{P} = \exp\bigg(- \frac{\mathcal{L}}{n} \bigg) = \exp\bigg(- \cfrac{1}{n}\sum_{d \in D}\sum_{w \in d} n_{dw} \ln \big(\sum_{t \in T}\phi_{wt}\theta_{td} \big)\bigg)$$

Это традиционная мера качества в тематическом моделировании, которая основана на правдоподобии модели $\mathcal{L}$. Число итераций $max\_iter$ в алгоритме обучения следует выбирать достаточным для того, чтобы перплексия перестала существенно убывать. Однако известно, что перплексия плохо отражает интерпретируемость найденных тем, поэтому помимо нее обычно используются дополнительные меры или экспертные оценки.

<a id='plsa_and_application'></a>
# 2. Реализация ЕМ-алгоритма для модели PLSA и его применение [[toc](#toc)]

<a id='load'></a>
## 2.1 Загрузка данных [[toc](#toc)]

Загрузите [коллекцию писем Х. Клинтон](https://www.dropbox.com/s/je8vq5fsb8xpy2u/hillary_data.zip?dl=0). 

Извлеките полные тексты писем из файла *Emails.csv* и подготовьте данные в формате "мешка слов" с помощью функции  [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) пакета sklearn. Рекомендуется произвести фильтрацию слов по частотности для удаления слишком редких и стоп-слов (рекомендованный нижний порог в пределах 10 и верхний 400-600).

In [ ]:
emails_path = './hillary_data/output/Emails.csv'
raw_data = pd.read_csv(emails_path)
emails = list(raw_data['RawText'])
vectorizer = CountVectorizer(min_df=10, max_df=500, stop_words='english')
DW = vectorizer.fit_transform(emails)

word2index = dict(vectorizer.vocabulary_)
index2word = {i:w for w,i in word2index.items()}
print('Document-word matrix size = {}'.format(DW.shape))
print('Dictionary size = {}'.format(len(word2index)))

In [ ]:
print('Number of empty documents:', np.sum(np.sum(DW, axis=1) == 0))
print('Number of never occuring terms:', np.sum(np.sum(DW, axis=0) == 0))

Как оказывается, есть такие тексты, векторное представление которых содержит только нули. Эти тексты нужно убрать, так как по существу они бессмысленны.

In [ ]:
print(DW.shape)
n_terms = DW.sum(axis=1).A1.flatten()
if np.any(n_terms == 0):
    DW = DW[np.flatnonzero(n_terms != 0)]
print(DW.shape)

In [ ]:
print('Number of empty documents:', np.sum(np.sum(DW, axis=1) == 0))
print('Number of never occuring terms:', np.sum(np.sum(DW, axis=0) == 0))

<a id='plsa'></a>
## 2.2 Реализация ЕМ-алгоритма [[toc](#toc)]

In [ ]:
class PLSA(Checker):
    def __init__(self, n_topics, init='rand', max_iter=10, verbose=0, seed=0,
                 eps=1e-30, check_errors=False, max_error=1e-10,
                 memory_limit=2048):
        super().__init__(max_error)
        self._n_topics = n_topics
        self._init = init
        self._seed = seed
        self._max_iter = max_iter
        self.verbose = verbose
        self._check_errors = check_errors
        self._eps = eps
        self._class_name = type(self).__name__
        self._memory_limit = memory_limit
        self._n_runs = 0
        
    def set_max_iter(self, max_iter):
        self._check_int(max_iter, 'max_iter')
        self._check_positive(max_iter, 'max_iter')
        self._max_iter = max_iter
        
    def __call__(self, DW, copy=False, continued=False):
        """
        Аргументы:
            :param DW - матрица документ-термин
            :type DW  - scipy.sparse.csr_matrix размера [n_documents, n_terms]
        """
        if continued:
            self._check_continuation(DW, continued)
        else:
            self._set_DW(DW, copy)
            self._initialize_phi_theta()
            self._initialize_params()
            self._determine_method()
        self._printers[3](self._make_msg('initialized'))
        output = self._method()
        self._n_runs += 1
        self._initial_iter = self._n_iter
        return output

    def _iterative(self):
        for n_iter in range(self._initial_iter, self._initial_iter + self._max_iter):
            self._n_iter = n_iter
            self.H_wt = np.zeros_like(self.Phi)                               # [W x T] +
            self._log_perplexity = 0
            for n_doc in range(self._n_docs):
                P_dwt = np.multiply(self.Phi, self.Theta[:, n_doc][None, :])  # [W x T] +
                P_dw  = np.maximum(np.sum(P_dwt, axis=1)[:, None], self._eps) # [W, 1]  +
                N_dw  = self.DW[n_doc].toarray().T                            # [W, 1]  +
                self._log_perplexity -= np.sum(N_dw * np.log(P_dw)) / self._n
                H_dwt = P_dwt / P_dw                                          # [W x T] +
                H_dwt = np.multiply(N_dw, H_dwt)                              # [W x T]
                self.H_wt += H_dwt                                            # [W x T]
                self.Theta[:, n_doc] = np.sum(H_dwt, axis=0) / self._n_d[n_doc]
                if self._check_errors:
                    self._check_distr(self.Theta[:, n_doc], 'Theta[:, {}]'.format(n_doc))
            self.Phi = self.H_wt / np.sum(self.H_wt, axis=0)[None, :]    # [W x T]
            if self._check_errors:
                for n_topic in range(self._n_topics):
                    self._check_dsitr(self.Phi[:, n_topic], 'Phi[:, {}]'.format(n_topic))
                self._printers[6]('n_iter = {}: errors checking activated: no errors')
            self._log_perplexities.append(self._log_perplexity)
            self._printers[1]('n_iter = {}: log_P = {}'.format(self._n_iter, self._log_perplexity))
        return self.Phi, self.Theta, self._log_perplexities
    
    def _direct(self):
        self.WD = self.DW.toarray().T             # [W x D] +
        self.Theta = self.Theta.T
        for n_iter in range(self._initial_iter, self._initial_iter + self._max_iter):
            self._n_iter = n_iter
            H_wdt = self.Phi.reshape((self._voc_size, 1, self._n_topics)) * \
                    self.Theta.reshape((1, self._n_docs, self._n_topics))  # [W x D x T] +
            H_wdt = np.maximum(H_wdt, self._eps)
            P_wd = np.sum(H_wdt, axis=2)           # [W x D] +
            self._log_perplexity = -np.sum(self.WD * np.log(P_wd)) / self._n
            H_wdt = H_wdt / P_wd[:, :, None]       # [W x D x T]
            H_wdt = self.WD[:, :, None] * H_wdt    # [W x D x T]
            self.Theta = H_wdt.sum(axis=0) / self._n_d[:, None]
            self.Phi = H_wdt.sum(axis=1)           # [W x T]
            self.Phi = self.Phi / self.Phi.sum(axis=0)[None, :]
            self._printers[1]('n_iter = {}: log_P = {}'.format(self._n_iter, self._log_perplexity))
        self.Theta = self.Theta.T
        return self.Phi, self.Theta, self._log_perplexities
            
    def _check_continuation(self, DW, continued):
        assert isinstance(DW, scipy.sparse.csr_matrix) 
        if continued:
            # Проверка того, что уже запускали
            assert self._n_runs > 0
            # Проверка того, что размер переданной матрицы совпадает с ранее исопользованными
            assert self._n_docs, self._voc_size == DW.shape

    def _set_DW(self, DW, copy):
        assert isinstance(DW, scipy.sparse.csr_matrix)
        if copy:
            self.DW = copy.deepcopy(DW)
        else:
            self.DW = DW
        self._n_d = self.DW.sum(axis=1).A1
        self._n   = np.sum(self._n_d)
        self._n_docs, self._voc_size = self.DW.shape

    def _initialize_params(self):
        self._log_perplexities = []
        self._n_iter = 0
        self._initial_iter = 0
        
    def _initialize_phi_theta(self):
        if self._init == 'rand':
            np.random.seed(self._seed)
            self.Phi = np.maximum(np.random.rand(self._voc_size, self._n_topics), self._eps)
            self.Theta = np.maximum(np.random.rand(self._n_topics, self._n_docs), self._eps)
            self.Phi = self.Phi / np.sum(self.Phi, axis=0)[None, :]
            self.Theta = self.Theta / np.sum(self.Theta, axis=0)[None, :]
        elif self._init == 'uniform':
            self.Phi = np.full((self._voc_size, self._n_topics), 1.0 / self._voc_size)
            self.Theta = np.full((self._n_topics, self._n_docs), 1.0 / self._n_topics)
        else:
            raise ValueError(self._make_msg('Unknown value of init', method_name))        
        
    def _determine_method(self):
        nbytes = self.DW.dtype.type(0).nbytes
        required_memory_direct = (self._n_topics * self._n_docs +     # self.Theta
                                  self._voc_size * self._n_topics +   # self.Phi
                                  self._voc_size * self._n_docs * self._n_topics + # H_wdt
                                  self._voc_size * self._n_docs) * nbytes * 1.0  # P_wd
        required_memory_iter   = (self._n_topics * self._n_docs +     # self.Theta
                                  self._voc_size * self._n_topics +   # self.Phi
                                  self._voc_size * self._n_topics +   # H_wt
                                  self._voc_size * self._n_topics +   # P_dwt
                                  self._voc_size * self._n_topics +   # H_dwt
                                  self._voc_size * 2) * nbytes * 1.0 # N_dw, P_dw
        required_memory_direct /= 2 ** 20
        required_memory_iter /= 2 ** 20
        self._printers[5]('Required memory direct = {} Mb'.format(required_memory_direct))
        self._printers[5]('Required memory iter   = {} Mb'.format(required_memory_iter))
        self._printers[5]('Memory limit   = {} Mb'.format(self._memory_limit))
        if required_memory_direct > self._memory_limit:
            self._printers[5]('Choosing iterative solving method')
            self._method = self._iterative
        else:
            self._printers[5]('Choosing direct sovling method')
            self._method = self._direct
        
    def _make_msg(self, msg, method_name=None):
        if method_name is None:
            return '{}:{}'.format(self._class_name, msg)
        return '{}:{}:{}'.format(self._class_name, method_name, msg)

<a id='plsa_task1'></a>
## 2.3 Задание 1 [3 балла] [[toc](#toc)]
Примените ваш алгоритм к подготовленным данным, рассмотрите число тем T = 5. Постройте график значения перплексии в зависимости от итерации (убедитесь в корректности реализации: график перплексии должен быть невозрастающим). Выведите для каждой темы топ-20 наиболее вероятных слов.

### 3.2.1.1  Применение алгоритма к подготовленным данным для числа тем T = 5. Построение графика зависимости значения перплексии в зависимости от итерации.

In [ ]:
plsa = PLSA(n_topics=5, init='rand', max_iter=100, verbose=11, seed=11)
Phi, Theta, log_perplexities = plsa(DW)

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(log_perplexities, marker='x', color='b', zorder=2)
plt.grid(linestyle='--', alpha=0.5)
plt.title('Perplexity vs PLSA iteration');
plt.xlabel('iteration');
plt.ylabel('$\log(\mathcal{P})$');

In [ ]:
#### 3.2.1.2 Вывод 20 наиболее вероятных слов для каждой темы.

In [ ]:
def get_max_indices(values, n_max):
    values = [[i, x] for i, x in enumerate(values)]
    values = sorted(values, key=lambda x: -x[1])
    indices = [i for i, x in values[:n_max]]
    return np.array(indices)

def get_representatives(Phi, n_max):
    representatives = pd.DataFrame()
    for topic in range(Phi.shape[1]):
        words = [index2word[i] for i in get_max_indices(Phi[:, topic], max_words)]
        representatives[topic] = words
    return representatives

In [ ]:
max_words = 20
representatives = get_representatives(Phi, max_words)
print(representatives)

<a id='plsa_task2'></a>
## 2.2 Задание 2 [0.5 балла] [[toc](#toc)]
Рассмотрите число тем T = 10, 20. Сравните между собой топ-20 наиболее вероятных слов в каждой теме (а также для модели, полученной ранее). Можно ли сказать, что конкретность каждой темы изменяется с ростом их числа?

### 2.2.1 Применение алгоритма для числа тем T = 10. Вывод топ-20 наиболее вероятных слов для каждой темы.

### 2.2.2 Применение алгоритма для числа тем T = 20. Вывод топ-20 наиболее вероятных слов для каждой темы.

В целом, с ростом числа тем растет интерпретируемость полученных результатов, т.е. растет все большему числу тем можно поставить в соответствие некую метку. Например, найденным выше группам топ-20 наиболее вероятных слов можно присвоить следующие метки: "политика", "дипломатия", "военные действия", "безопасность", "Персидский Залив", "гуманитарная помощь", "проживаение и путешествие" (тема 6 для T = 20) и т.п.

<a id='plsa_task3'></a>
## 2.3 Задание 3 [0.5 балла] [[toc](#toc)]
Протестируйте модель для разных начальных приближений. Что можно сказать об устойчивости алгоритма (идентичны ли топовые слова в соответствующих темах моделей)?

Ниже модель запускается для 4-х различных значений "зерна" генератора случайных чисел. Число тем равно 10, а число итераций выбрано равным 40. Во всех случаях топовые слова в темах совпали. Таким образом, можно утверждать, что алгоритм PLSA достаточно устойчив к выбору начальных приближений.

<a id='lda_and_visualization'></a>
# 3. Модель LDA и визуализация [[toc](#toc)]

Модель [LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) является наиболее популярной тематической моделью. Единственное отличие от модели PLSA заключается в введении априорных распределений Дирихле на столбцы матриц $\Phi$ и $\Theta$, которое может способствовать дополнительному сглаживанию или разреживанию параметров.

В этом задании предлагается воспользоваться реализацией модели [LdaModel](https://radimrehurek.com/gensim/models/ldamodel.html), обучение которой основано на вариационном байесовском выводе. Для выполнения задания вам потребуется установить пакеты [gensim](https://radimrehurek.com/gensim/install.html) и [pyldavis 2.0](https://pyldavis.readthedocs.io/en/latest/readme.html#installation).

Подготовьте данные в формате, подходящем для *gensim* (полное API gensim можно найти [здесь](https://radimrehurek.com/gensim/apiref.html)). Пример обработки вывода *CountVectorizer* для gensim можно найти [здесь](https://gist.github.com/aronwc/8248457).

<a id='lda_task1'></a>
## 3.1 Задание 1 [1 балл] [[toc](#toc)]

Примените [LdaModel](https://radimrehurek.com/gensim/models/ldamodel.html) к подготовленным данным (рекомендуется задать заведомо большое число итераций в параметре *passes*, например, 30). Визуально сравните полученные темы по топ-20 наиболее вероятным словам с темами, полученными вашей реализацией ЕМ-алгоритма. У какой из моделей получились более интерпретируемые темы?

In [ ]:
from gensim import matutils
from gensim.models.ldamodel import LdaModel

emails_path = './hillary_data/output/Emails.csv'
raw_data = pd.read_csv(emails_path)
emails = list(raw_data['RawText'])
vectorizer = CountVectorizer(min_df=10, max_df=500, stop_words='english')
DW = vectorizer.fit_transform(emails)

word2index = dict(vectorizer.vocabulary_)
index2word = {i:w for w,i in word2index.items()}
print('Document-word matrix size = {}'.format(DW.shape))
print('Dictionary size = {}'.format(len(word2index)))
num_topics = 10
passes = 40

corpus = matutils.Sparse2Corpus(DW.tocoo().transpose().tocsc())
lda = LdaModel(corpus, num_topics=num_topics, passes=passes, id2word=index2word)

In [ ]:
def get_lda_representatives(lda, index2word, n):
    representatives = pd.DataFrame()
    topics = lda.show_topics(num_topics=-1, num_words=n, formatted=False)
    for ti, topic in topics:
        words = [w for w, p in topic]
        representatives[ti] = words
    return representatives

max_words = 20
representatives = get_lda_representatives(lda, index2word, max_words)
print(representatives)

В целом, оба алгоритма PLSA и LDA выделяют одни и те же темы. Стоит отметить, что какие-то темы лучше выделены в результате применения PLSA, в то время как другие - в результате применения LDA. Однако существенных различий не наблюдается, что может быть связано с достаточно большим размером коллекции текстов.

<a id='lda_task2'></a>
## 3.2 Задание  2 [1 балл] [[toc](#toc)]
Визуализируйте модель из gensim с помощью ldavis (описание API LDAvis для работы с gensim есть [здесь](http://pyldavis.readthedocs.io/en/latest/modules/API.html)), пример — [здесь](https://github.com/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb).

In [ ]:
topic_term_matrix = lda.get_topics()
print(topic_term_matrix.shape)

doc_topic_matrix = np.zeros((DW.shape[0], num_topics))
for n_doc in tqdm(range(DW.shape[0])):
    dw = DW[n_doc]
    bow = []
    for n_row in dw.nonzero()[1]:
        bow.append((n_row, dw[0, n_row]))
    doc_topic_list = lda.get_document_topics(bow)
    for topic, prob in doc_topic_list:
        doc_topic_matrix[n_doc, topic] = prob
print(doc_topic_matrix.shape)

In [ ]:
model_data = {}
doc_topic_matrix /= np.sum(doc_topic_matrix, axis=1)[:, None]
model_data['topic_term_dists'] = topic_term_matrix
model_data['doc_topic_dists'] = doc_topic_matrix
model_data['doc_lengths'] = DW.sum(axis=1).A1.flatten()
model_data['term_frequency'] = DW.sum(axis=0).A1.flatten()
model_data['vocab'] = [index2word[i] for i in range(DW.shape[1])]

In [ ]:
import pyLDAvis
vis_data = pyLDAvis.prepare(**model_data)

In [ ]:
pyLDAvis.display(vis_data)

In [ ]:
vis_data = pyLDAvis.prepare(mds='tsne', **model_data)
pyLDAvis.display(vis_data)

In [ ]:
pyLDAvis.prepare(mds='mmds', **model_data)
pyLDAvis.display(vis_data)

In [ ]:
import gensim
import pyLDAvis.gensim
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, index2word)
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

### Рекомендации к выполнению [[toc](#toc)]
Для обучения *LdaModel* и её последующей визуализации потребуется словарь формата gensim, который можно получить следующей командой

    dictionary = gensim.corpora.Dictionary.from_corpus(corpora, vocab_dict)

где *corpora* содержит полученное с помощью gensim представление коллекции, а *vocab_dict* — это dict, полученный после работы CountVectorizer, ставящий в соответствие каждому номеру строки в матрице данных само слово в виде строки.